Testes

In [ ]:
dfSentimentScore.createTempView('thais')

In [ ]:
thaisDF = dfSentimentScore.toPandas()
print(thaisDF)

In [ ]:
udfSentiment = udf(getSentiment, IntegerType())
dfSentiment = dfSentimentScore.withColumn('sentiment', udfSentiment(col('sentimentScore')))
dfSentiment.select('filteredCleanedTweetText','sentimentScore','sentiment').show(5)

In [ ]:
sentiment_analysis_udf = udf(perform_sentiment_analysis, StringType())

In [ ]:
result_df = df_result.withColumn("sentiment_score", sentiment_analysis_udf(col("stop")))


In [ ]:
result_df.select(col('stop')).show()

In [ ]:
type(result_df)

In [ ]:
new_df = df_result.withColumn("stop", analyzerSentiment(col('stop')))

In [ ]:
new_df = df_result.withColumn("stop", analyzerSentiment(col('stop')))
new_df.show()

In [ ]:
analyze_sentiment = udf(lambda text: int(sentiment_analyzer.polarity_scores(text)["pos"] >
0.5), IntegerType())

In [ ]:
analyze_sentiment

In [ ]:
#@udf
#def to_upper(text):
#    return text.upper()

#new_df = sdf.withColumn("name", to_upper(col('name')))
#new_df.show()

for col_name in df_result.columns:
    new_df = df_result = df_result.withColumn(col_name, analyzerSentiment(col(col_name)))

In [ ]:
new_df.select(col('stop')).show()

In [ ]:
df_result.mapInArrow(analyzer.polarity_scores,df_result.schema).show()

In [ ]:
scores.toPandas()

In [ ]:
# review = "With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent."
scores = analyzer.polarity_scores(df_result)
print(scores)

In [ ]:
def cleanTweet(txt):
    '''
    Remove twitter return handles (RT @xxx:)
    '''
    txt = removePattern(txt, 'RT @[\w]*:')
    '''
    Remove twitter handles (@xxx)
    '''
    txt = removePattern(txt, '@[\w]*')
    '''
    Remove URL links (httpxxx)
    '''
    txt = removePattern(txt, 'https?://[A-Za-z0-9./]*')
    '''
    Remove special characters, numbers, punctuations
    '''
    txt = re.sub('[^A-Za-z]+', ' ', txt)
    return txt

In [ ]:
tweet_v1 = result2.withColumn("stop", F.regexp_replace("stop", r"RT @[\w]*:", ""))
tweet_v2 = result2.withColumn("stop", F.regexp_replace("stop", r"@[\w]*", ""))
tweet_final = result2.withColumn("stop", F.regexp_replace("stop", r"https?://[A-Za-z0-9./]", ""))


In [ ]:
result2.select(col('stop')).show(truncate = False)

In [ ]:
result2_v2.select(col('stop')).show(truncate = False)

In [ ]:
def removePattern(inputText, pattern):
    r = re.findall(pattern, inputText)
    for i in r:
        inputText = re.sub(i, '', inputText)        
    return inputText

def cleanTweet(txt):
    '''
    Remove twitter return handles (RT @xxx:)
    '''
    txt = removePattern(txt, 'RT @[\w]*:')
    '''
    Remove twitter handles (@xxx)
    '''
    txt = removePattern(txt, '@[\w]*')
    '''
    Remove URL links (httpxxx)
    '''
    txt = removePattern(txt, 'https?://[A-Za-z0-9./]*')
    '''
    Remove special characters, numbers, punctuations
    '''
    txt = re.sub('[^A-Za-z]+', ' ', txt)
    return txt

def getCleanTweetText(filteredTweetText):
    return ' '.join(filteredTweetText)

def getSentimentScore(tweetText):
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(tweetText)
    return float(vs['compound'])

def getSentiment(score):
    return 1 if score > 0 else 0


def getTweetArray(tweet):
    return tweet.split(' ')

In [ ]:
dados2 = (spSession.read
      .option("multiline", "true") # alguns registros estão em mais de uma linha
      .option("quote", '"') # informando que temos as aspas para separar o texto (q tem virgula dentro) para nao impactar o delimitador
      .option("header", "true") 
      #.option("mode", "DROPMALFORMED")
      .option("escape", '"') # separando caracteres especiais
      .csv('dados/pandas_v2.csv')
)

In [ ]:
udfCleanTweet = udf(cleanTweet, StringType())
dfCleanTweet= dados2.withColumn('cleanTweetText', udfCleanTweet('text'))
dfCleanTweet.select('text','cleanTweetText').show(5)

In [ ]:
tokenizer = Tokenizer(inputCol='cleanTweetText', outputCol='words')
dfCleanTweetTokenized = tokenizer.transform(dfCleanTweet)
dfCleanTweetTokenized.select('text','cleanTweetText','words').show(5)

In [ ]:
remover = StopWordsRemover(inputCol='words', outputCol='filteredTweetText')
dfStopwordRemoved=remover.transform(dfCleanTweetTokenized)
dfStopwordRemoved.select('text','cleanTweetText','words','filteredTweetText').show(truncate = False)

In [ ]:
dfStopwordRemoved.select('filteredTweetText').show(truncate = False)

In [ ]:
udfCleanTweetText = udf(getCleanTweetText, StringType())
dfFilteredCleanedTweet = dfStopwordRemoved.withColumn('filteredCleanedTweetText', udfCleanTweetText('filteredTweetText'))
dfFilteredCleanedTweet.select('filteredCleanedTweetText').show(5)

In [ ]:
udfSentimentScore = udf(getSentimentScore, FloatType())
dfSentimentScore = dfFilteredCleanedTweet.withColumn('sentimentScore', udfSentimentScore('filteredCleanedTweetText'))
dfSentimentScore.select('filteredCleanedTweetText','sentimentScore').show(5)

In [ ]:
udfSentiment = udf(getSentiment, IntegerType())
dfSentiment = dfSentimentScore.withColumn('sentiment', udfSentiment('sentimentScore'))
dfSentiment.select('filteredCleanedTweetText','sentimentScore','sentiment').show(5)

In [ ]:
dfSentiment.groupBy('sentiment').count().show()

dfPlotVaderSentiment=dfSentiment.groupBy('sentiment').count().toPandas()
dfPlotVaderSentiment